In [1]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/92/2d/880fcd65e4414b05088193e6f2cfb86fdf90003dd2dd0f4d1bc465348f0e/tensorflow-2.15.0-cp311-cp311-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.1 MB/s eta 0:00:00
  Obtaining dependency information for libclang>=13.0.0 from https://files.pythonhosted.org/packages/c9/ea/fe2a69cc6cfebf7c7ee8a6357566fc1cbb91632bde5869b669a396accb5f/libclang-16.0.6-py2.py3-none-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for

In [2]:
import os
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Download the required NLTK packages
nltk.download('punkt')

def extract_paragraphs_from_files(folder):
    paragraphs = []
    # List all files in the directory
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        # Check if it is a file
        if os.path.isfile(filepath):
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                text = file.read()
                # Split text into paragraphs
                paras = text.split('\n\n')
                # Filter out any empty paragraphs
                paras = [para for para in paras if para and not para.isspace()]
                paragraphs.extend(paras)
    return paragraphs

# Extract paragraphs from both folders
gpt_paragraphs = extract_paragraphs_from_files('CowboyChatGPT')
human_paragraphs = extract_paragraphs_from_files('CowboyHuman')

# Label the paragraphs (1 for GPT, 0 for Human)
gpt_labels = [1] * len(gpt_paragraphs)
human_labels = [0] * len(human_paragraphs)

# Combine the paragraphs and labels
paragraphs = gpt_paragraphs + human_paragraphs
labels = gpt_labels + human_labels

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(paragraphs)
sequences = tokenizer.texts_to_sequences(paragraphs)
X = pad_sequences(sequences)

# Convert the labels into a numpy array
y = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Neural network model with doubled size
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256, input_length=X.shape[1]))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model on the test set
_, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Predictions on the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_binary)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Human', 'GPT'], yticklabels=['Human', 'GPT'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
class_report = classification_report(y_test, y_pred_binary, target_names=['Human', 'GPT'])
print(class_report)

# Save the trained model to a file
# Save the trained model to a file in the native Keras format
model.save('nn_twice_as_big_model.keras')

2023-11-28 10:32:18.323275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/100
2/2 [==============================] - 3s 990ms/step - loss: 0.6764 - accuracy: 0.7414 - val_loss: 0.5766 - val_accuracy: 0.8000
Epoch 2/100
2/2 [==============================] - 2s 766ms/step - loss: 0.6007 - accuracy: 0.7414 - val_loss: 0.5004 - val_accuracy: 0.8000
Epoch 3/100
2/2 [==============================] - 2s 875ms/step - loss: 0.5681 - accuracy: 0.7414 - val_loss: 0.4530 - val_accuracy: 0.8000
Epoch 4/100
2/2 [==============================] - 2s 855ms/step - loss: 0.5414 - accuracy: 0.7414 - val_loss: 0.4219 - val_accuracy: 0.8000
Epoch 5/100
2/2 [==============================] - 1s 747ms/step - loss: 0.5249 - accuracy: 0.7414 - val_loss: 0.3983 - val_accuracy: 0.8000
Epoch 6/100
2/2 [==============================] - 1s 800ms/step - loss: 0.5136 - accuracy: 0.7414 - val_loss: 0.3762 - val_accuracy: 0.8000
Epoch 7/100
2/2 [==============================] - 2s 730ms/step - loss: 0.4980 - accuracy: 0.7414 - val_loss: 0.3618 - val_accuracy: 0.8000
Epoch 8/100
2


KeyboardInterrupt

